<a href="https://colab.research.google.com/github/TallB3/valuebell_analytics/blob/main/v_2_valuebell_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shows

In [ ]:
# Create shows

import requests
import pandas as pd

# Your API key
api_key = "UHZPM4uIejLi4YKMueqkHg"

# Base URL
url = "https://api.transistor.fm/v1/shows"

# Initialize variables for pagination
page = 1
shows_list = []

while True:
    # Request with pagination
    response = requests.get(
        url,
        headers={"X-Api-Key": api_key},
        params={"pagination[page]": page, "pagination[per]": 10}
    )

    data = response.json()

    # Break the loop if no more shows are available
    if not data['data']:
        break

    # Extract show id, title, and feed_url from each response
    for show in data['data']:
        show_id = show['id']
        title = show['attributes']['title']
        feed_url = show['attributes']['feed_url']
        shows_list.append({"Show ID": show_id, "Title": title, "Feed URL": feed_url})

    # Move to the next page
    page += 1

# Create a DataFrame named 'shows'
shows = pd.DataFrame(shows_list)

# Fix types
shows["Show ID"] = pd.to_numeric(shows["Show ID"], errors='coerce')

# Display the DataFrame
shows


,Show ID,Title,Feed URL
0,44556,The Business Mindset,https://feeds.transistor.fm/the-business-mindset
1,57522,Change Matters,https://feeds.transistor.fm/change-matters
2,54502,שימו לב | פודקאסט מציל חיים,https://feeds.transistor.fm/valuebell-simu-lev
3,37413,רונית רמר מספרת ׳סיפורי סיפורים׳,https://feeds.transistor.fm/valuebell-ronit-remer
4,32847,מנהלים ועובדים על זה || עו״ד יניב אופק,https://feeds.transistor.fm/yaniv-ofek
5,36996,סוד המצוינות הישראלי - גיא קצוביץ,https://feeds.transistor.fm/valuebell-guy-book
6,50986,דואט | דודו ונטלי אהרון,https://feeds.transistor.fm/valuebell-dudu-aharon
7,32621,לנצח את הפסיכומטרי - תמיר עגיב,https://feeds.transistor.fm/valuebell-psychome...
8,31438,ליגלייז - פודקאסט קנאביס,https://feeds.transistor.fm/valuebell-legalize
9,43328,לא בטעות ולא במקרה || אלאיה הוף,https://feeds.transistor.fm/valuebell-alayah-hoff


# Episodes

In [ ]:
# Create episodes_df

import requests
import pandas as pd
import time

# Your API key
api_key = "UHZPM4uIejLi4YKMueqkHg"

# Base URL for episodes
url = "https://api.transistor.fm/v1/episodes"

# List to store all episodes from all shows
all_episodes = []

# Loop through each show in the 'shows' DataFrame
for index, row in shows.iterrows():
    show_id = row['Show ID']
    page = 1

    while True:
        # Make the API request for the current show and page
        response = requests.get(
            url,
            headers={"X-Api-Key": api_key},
            params={
                "show_id": show_id,
                "pagination[page]": page,
                "pagination[per]": 10  # Adjust per request
            }
        )

        # Convert response to JSON
        try:
            data = response.json()
        except ValueError:
            print(f"Error parsing JSON response for show ID {show_id}. Response text: {response.text}")
            break  # Break the loop if we cannot parse the response

        # Check for rate limit error (HTTP status code 429)
        if response.status_code == 429:
            print(f"Rate limit exceeded for show ID {show_id}. Waiting for 10 seconds...")
            time.sleep(12)  # Wait for 10 seconds before retrying
            continue

        # Check if 'data' exists in the response
        if 'data' not in data:
            print(f"No 'data' field in response for show ID {show_id}. Response: {data}")
            break  # No data, stop processing this show

        # Check if there are any episodes
        if not data['data']:
            break  # No more episodes, move to the next show

        # Loop through the episodes and extract the required fields
        for episode in data['data']:
            episode_attributes = episode['attributes']
            all_episodes.append({
                "Show ID": show_id,
                "Show Title": row['Title'],  # Include show title from the 'shows' DataFrame
                "Episode ID": episode['id'],
                "Title": episode_attributes.get('title'),
                "Number": episode_attributes.get('number'),
                "Season": episode_attributes.get('season'),
                "Status": episode_attributes.get('status'),
                "Published At": episode_attributes.get('published_at'),
                "Created At": episode_attributes.get('created_at'),
                "Duration (seconds)": episode_attributes.get('duration'),
                "Type": episode_attributes.get('type'),
            })

        # Move to the next page
        page += 1

        # Pause briefly between requests to avoid rate limit
        time.sleep(2)

# Create a DataFrame for all episodes from all shows
episodes_df = pd.DataFrame(all_episodes)

# Convert the date columns to datetime format
episodes_df['Published At'] = pd.to_datetime(episodes_df['Published At'])
episodes_df['Created At'] = pd.to_datetime(episodes_df['Created At'])

# Display the DataFrame
episodes_df


,Show ID,Show Title,Episode ID,Title,Number,Season,Status,Published At,Created At,Duration (seconds),Type
0,44556,The Business Mindset,2124642,פרק שלישי 8.10.24,19.0,1,draft,2024-10-09 10:13:09.783000+00:00,2024-10-09 10:13:09.784000+00:00,3083.0,full
1,44556,The Business Mindset,2124641,8.10.24 פרק שני,18.0,1,draft,2024-10-09 10:11:03.191000+00:00,2024-10-09 10:11:03.192000+00:00,2731.0,full
2,44556,The Business Mindset,2124639,8.10.24 פרק ראשון,17.0,1,draft,2024-10-09 10:09:14.381000+00:00,2024-10-09 10:09:14.382000+00:00,2200.0,full
3,44556,The Business Mindset,2119848,"נעמי סחייק | נחישות, ערכיות וחיוביות",16.0,1,published,2024-10-06 12:03:54.890000+00:00,2024-10-06 12:03:54.891000+00:00,1992.0,full
4,44556,The Business Mindset,2099897,"חני ויינברגר | ערכים, אהבה ואמונה",15.0,1,published,2024-09-21 17:56:16.207000+00:00,2024-09-21 17:54:34.533000+00:00,2405.0,full
...,...,...,...,...,...,...,...,...,...,...,...
334,31732,Digital Health Business,960116,Dr. John Danaher || The Training and Education...,3.0,1,published,2022-08-29 13:00:00+00:00,2022-07-25 15:52:30.117000+00:00,2145.0,full
335,31732,Digital Health Business,960119,"(Hebrew) Prof. Eyal Zimlichman, MD || The Isra...",4.0,1,published,2022-08-01 14:53:56.487000+00:00,2022-07-25 15:57:05.497000+00:00,1855.0,full
336,31732,Digital Health Business,919241,Deneen Vojta || The Payers’ Perspective,2.0,1,published,2022-07-18 13:00:00+00:00,2022-06-15 10:06:14.532000+00:00,1680.0,full
337,31732,Digital Health Business,921102,Dr. David Shulkin || The US Government Perspec...,1.0,1,published,2022-07-04 03:00:00+00:00,2022-06-16 09:10:16.704000+00:00,1609.0,full


# Downloads

In [ ]:
# Create Downloads df

import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Your API key
api_key = "UHZPM4uIejLi4YKMueqkHg"

# Base URL for episode analytics
analytics_url = "https://api.transistor.fm/v1/analytics/{}/episodes"

# Create an empty list to store downloads data
downloads_list = []

# Current date (end date for the range)
end_date = (datetime.now() - timedelta(days=1)).strftime("%d-%m-%Y")

# Iterate over each show in the 'shows' DataFrame
for index, row in shows.iterrows():
    show_id = row['Show ID']

    # Get the earliest creation date for episodes of this show from episodes_df
    show_episodes = episodes_df[episodes_df['Show ID'] == show_id]
    if not show_episodes.empty:
        start_date = pd.to_datetime(show_episodes['Created At']).min().strftime("%d-%m-%Y")
    else:
        print(f"No episodes found for show {show_id}. Skipping.")
        continue  # Skip if no episodes are found for this show

    # Retry mechanism for rate limit handling
    while True:
        try:
            # Make the API request for episode downloads analytics
            response = requests.get(
                analytics_url.format(show_id),
                headers={"X-Api-Key": api_key},
                params={
                    "start_date": start_date,
                    "end_date": end_date
                }
            )

            # Check for rate limit exceeded error (HTTP status 429)
            if response.status_code == 429:
                print(f"Rate limit exceeded for show {show_id}. Waiting for 12 seconds before retrying...")
                time.sleep(12)
                continue  # Retry the same request after the wait period

            # Convert response to JSON
            data = response.json()

            # Check if the 'data' field exists in the response
            if 'data' not in data:
                print(f"No 'data' field found for show {show_id}")
                break

            if 'attributes' not in data['data']:
                print(f"No 'attributes' field found in 'data' for show {show_id}")
                break

            if 'episodes' not in data['data']['attributes']:
                print(f"No 'episodes' field found in 'attributes' for show {show_id}")
                break

            # Extract download data for each episode
            for episode in data['data']['attributes']['episodes']:
                episode_id = episode['id']
                episode_title = episode['title']

                for download_data in episode['downloads']:
                    download_date = download_data['date']
                    download_count = download_data['downloads']

                    downloads_list.append({
                        "Episode ID": episode_id,
                        "Episode Title": episode_title,
                        "Show ID": show_id,
                        "Download Date": download_date,
                        "Downloads": download_count
                    })

            break  # Break out of the retry loop after successful data retrieval
        except Exception as e:
            print(f"An error occurred for show {show_id}: {e}")
            break  # Exit loop if an error occurs
        finally:
            time.sleep(2)  # Pause for 2 seconds between each request

# Create a DataFrame for the downloads data
downloads_df = pd.DataFrame(downloads_list)

# Change to date type
downloads_df["Download Date"] = pd.to_datetime(downloads_df["Download Date"], errors='coerce')

# Remove irrelevant rows

# First, merge 'Published At' from episodes_df into downloads_df using 'Episode ID'
# Convert 'Episode ID' to numeric type before merging, handling potential errors
downloads_df['Episode ID'] = pd.to_numeric(downloads_df['Episode ID'], errors='coerce')
episodes_df['Episode ID'] = pd.to_numeric(episodes_df['Episode ID'], errors='coerce')
downloads_df = pd.merge(downloads_df, episodes_df[['Episode ID', 'Published At', 'Created At']], on='Episode ID')

# Now create a new columns 'Publish Date' and 'Creation Date'
downloads_df['Publish Date'] = downloads_df['Published At'].dt.date
downloads_df['Creation Date'] = downloads_df['Created At'].dt.date

# Convert 'Publish Date' and 'Creation Date' to datetime64[ns] (without time component)
downloads_df['Publish Date'] = pd.to_datetime(downloads_df['Publish Date'])
downloads_df['Creation Date'] = pd.to_datetime(downloads_df['Creation Date'])

# Drop irrelevant columns and rows from downloads_df, reset indices
downloads_df = downloads_df[(downloads_df["Download Date"] >= downloads_df["Creation Date"]) | (downloads_df["Downloads"] > 0)]
downloads_df = downloads_df.drop(['Published At', 'Created At', 'Publish Date', 'Creation Date'], axis=1)
downloads_df = downloads_df.reset_index(drop=True)

# Display the DataFrame
downloads_df


No episodes found for show 53532. Skipping.


,Episode ID,Episode Title,Show ID,Download Date,Downloads
0,2119848,"נעמי סחייק | נחישות, ערכיות וחיוביות",44556,2024-10-07,29
1,2119848,"נעמי סחייק | נחישות, ערכיות וחיוביות",44556,2024-10-06,27
2,2099897,"חני ויינברגר | ערכים, אהבה ואמונה",44556,2024-10-07,6
3,2099897,"חני ויינברגר | ערכים, אהבה ואמונה",44556,2024-10-06,9
4,2099897,"חני ויינברגר | ערכים, אהבה ואמונה",44556,2024-10-05,3
...,...,...,...,...,...
154416,921102,Dr. David Shulkin || The US Government Perspec...,31732,2022-06-20,3
154417,921102,Dr. David Shulkin || The US Government Perspec...,31732,2022-06-19,1
154418,921102,Dr. David Shulkin || The US Government Perspec...,31732,2022-06-18,0
154419,921102,Dr. David Shulkin || The US Government Perspec...,31732,2022-06-17,2


# All Together!

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# Your API key
api_key = "UHZPM4uIejLi4YKMueqkHg"

# Create shows

# Base URL
url = "https://api.transistor.fm/v1/shows"

# Initialize variables for pagination
page = 1
shows_list = []

while True:
    # Request with pagination
    response = requests.get(
        url,
        headers={"X-Api-Key": api_key},
        params={"pagination[page]": page, "pagination[per]": 10}
    )

    data = response.json()

    # Break the loop if no more shows are available
    if not data['data']:
        break

    # Extract show id, title, and feed_url from each response
    for show in data['data']:
        show_id = show['id']
        title = show['attributes']['title']
        feed_url = show['attributes']['feed_url']
        shows_list.append({"Show ID": show_id, "Title": title, "Feed URL": feed_url})

    # Move to the next page
    page += 1

# Create a DataFrame named 'shows'
shows = pd.DataFrame(shows_list)

# Fix types
shows["Show ID"] = pd.to_numeric(shows["Show ID"], errors='coerce')

# Create episodes_df

# Base URL for episodes
url = "https://api.transistor.fm/v1/episodes"

# List to store all episodes from all shows
all_episodes = []

# Loop through each show in the 'shows' DataFrame
for index, row in shows.iterrows():
    show_id = row['Show ID']
    page = 1

    while True:
        # Make the API request for the current show and page
        response = requests.get(
            url,
            headers={"X-Api-Key": api_key},
            params={
                "show_id": show_id,
                "pagination[page]": page,
                "pagination[per]": 10  # Adjust per request
            }
        )

        # Convert response to JSON
        try:
            data = response.json()
        except ValueError:
            print(f"Error parsing JSON response for show ID {show_id}. Response text: {response.text}")
            break  # Break the loop if we cannot parse the response

        # Check for rate limit error (HTTP status code 429)
        if response.status_code == 429:
            print(f"Rate limit exceeded for show ID {show_id}. Waiting for 10 seconds...")
            time.sleep(12)  # Wait for 10 seconds before retrying
            continue

        # Check if 'data' exists in the response
        if 'data' not in data:
            print(f"No 'data' field in response for show ID {show_id}. Response: {data}")
            break  # No data, stop processing this show

        # Check if there are any episodes
        if not data['data']:
            break  # No more episodes, move to the next show

        # Loop through the episodes and extract the required fields
        for episode in data['data']:
            episode_attributes = episode['attributes']
            all_episodes.append({
                "Show ID": show_id,
                "Show Title": row['Title'],  # Include show title from the 'shows' DataFrame
                "Episode ID": episode['id'],
                "Title": episode_attributes.get('title'),
                "Number": episode_attributes.get('number'),
                "Season": episode_attributes.get('season'),
                "Status": episode_attributes.get('status'),
                "Published At": episode_attributes.get('published_at'),
                "Created At": episode_attributes.get('created_at'),
                "Duration (seconds)": episode_attributes.get('duration'),
                "Type": episode_attributes.get('type'),
            })

        # Move to the next page
        page += 1

        # Pause briefly between requests to avoid rate limit
        time.sleep(2)

# Create a DataFrame for all episodes from all shows
episodes_df = pd.DataFrame(all_episodes)

# Convert the date columns to datetime format
episodes_df['Published At'] = pd.to_datetime(episodes_df['Published At'])
episodes_df['Created At'] = pd.to_datetime(episodes_df['Created At'])

# Create downloads_df

# Base URL for episode analytics
analytics_url = "https://api.transistor.fm/v1/analytics/{}/episodes"

# Create an empty list to store downloads data
downloads_list = []

# Current date (end date for the range)
end_date = (datetime.now() - timedelta(days=1)).strftime("%d-%m-%Y")

# Iterate over each show in the 'shows' DataFrame
for index, row in shows.iterrows():
    show_id = row['Show ID']

    # Get the earliest creation date for episodes of this show from episodes_df
    show_episodes = episodes_df[episodes_df['Show ID'] == show_id]
    if not show_episodes.empty:
        start_date = pd.to_datetime(show_episodes['Created At']).min().strftime("%d-%m-%Y")
    else:
        print(f"No episodes found for show {show_id}. Skipping.")
        continue  # Skip if no episodes are found for this show

    # Retry mechanism for rate limit handling
    while True:
        try:
            # Make the API request for episode downloads analytics
            response = requests.get(
                analytics_url.format(show_id),
                headers={"X-Api-Key": api_key},
                params={
                    "start_date": start_date,
                    "end_date": end_date
                }
            )

            # Check for rate limit exceeded error (HTTP status 429)
            if response.status_code == 429:
                print(f"Rate limit exceeded for show {show_id}. Waiting for 12 seconds before retrying...")
                time.sleep(12)
                continue  # Retry the same request after the wait period

            # Convert response to JSON
            data = response.json()

            # Check if the 'data' field exists in the response
            if 'data' not in data:
                print(f"No 'data' field found for show {show_id}")
                break

            if 'attributes' not in data['data']:
                print(f"No 'attributes' field found in 'data' for show {show_id}")
                break

            if 'episodes' not in data['data']['attributes']:
                print(f"No 'episodes' field found in 'attributes' for show {show_id}")
                break

            # Extract download data for each episode
            for episode in data['data']['attributes']['episodes']:
                episode_id = episode['id']
                episode_title = episode['title']

                for download_data in episode['downloads']:
                    download_date = download_data['date']
                    download_count = download_data['downloads']

                    downloads_list.append({
                        "Episode ID": episode_id,
                        "Episode Title": episode_title,
                        "Show ID": show_id,
                        "Download Date": download_date,
                        "Downloads": download_count
                    })

            break  # Break out of the retry loop after successful data retrieval
        except Exception as e:
            print(f"An error occurred for show {show_id}: {e}")
            break  # Exit loop if an error occurs
        finally:
            time.sleep(2)  # Pause for 2 seconds between each request

# Create a DataFrame for the downloads data
downloads_df = pd.DataFrame(downloads_list)

# Change to date type
downloads_df["Download Date"] = pd.to_datetime(downloads_df["Download Date"], errors='coerce')

# Remove irrelevant rows

# First, merge 'Published At' from episodes_df into downloads_df using 'Episode ID'
# Convert 'Episode ID' to numeric type before merging, handling potential errors
downloads_df['Episode ID'] = pd.to_numeric(downloads_df['Episode ID'], errors='coerce')
episodes_df['Episode ID'] = pd.to_numeric(episodes_df['Episode ID'], errors='coerce')
downloads_df = pd.merge(downloads_df, episodes_df[['Episode ID', 'Published At', 'Created At']], on='Episode ID')

# Now create a new columns 'Publish Date' and 'Creation Date'
downloads_df['Publish Date'] = downloads_df['Published At'].dt.date
downloads_df['Creation Date'] = downloads_df['Created At'].dt.date

# Convert 'Publish Date' and 'Creation Date' to datetime64[ns] (without time component)
downloads_df['Publish Date'] = pd.to_datetime(downloads_df['Publish Date'])
downloads_df['Creation Date'] = pd.to_datetime(downloads_df['Creation Date'])

# Drop irrelevant columns and rows from downloads_df, reset indices
downloads_df = downloads_df[(downloads_df["Download Date"] >= downloads_df["Creation Date"]) | (downloads_df["Downloads"] > 0)]
downloads_df = downloads_df.drop(['Published At', 'Created At', 'Publish Date', 'Creation Date'], axis=1)
downloads_df = downloads_df.reset_index(drop=True)

# Adding changes to make insertions to SQL

# Rename the DataFrame columns to match the SQL table column names
shows.columns = ['show_id', 'title', 'rss_feed']

# Step 1: Drop the 'Show Title' column
episodes_df = episodes_df.drop(columns=['Show Title'])

# Step 2: Reorder the columns to match the SQL table's structure
episodes_df = episodes_df[['Episode ID', 'Show ID', 'Title', 'Number', 'Season', 'Status',
                           'Published At', 'Created At', 'Duration (seconds)', 'Type']]

# Step 3: Rename the columns to match SQL column names
episodes_df.columns = ['episode_id', 'show_id', 'title', 'number', 'season',
                       'status', 'publish_date', 'creation_date', 'duration_seconds', 'type']

downloads_df.drop(columns=['Episode Title', 'Show ID'], inplace=True)
downloads_df.columns = ['episode_id', 'download_date', 'downloads']


No episodes found for show 53532. Skipping.


# Checks and such

In [ ]:
shows

,show_id,title,rss_feed
0,44556,The Business Mindset,https://feeds.transistor.fm/the-business-mindset
1,57522,Change Matters,https://feeds.transistor.fm/change-matters
2,54502,שימו לב | פודקאסט מציל חיים,https://feeds.transistor.fm/valuebell-simu-lev
3,37413,רונית רמר מספרת ׳סיפורי סיפורים׳,https://feeds.transistor.fm/valuebell-ronit-remer
4,32847,מנהלים ועובדים על זה || עו״ד יניב אופק,https://feeds.transistor.fm/yaniv-ofek
5,36996,סוד המצוינות הישראלי - גיא קצוביץ,https://feeds.transistor.fm/valuebell-guy-book
6,50986,דואט | דודו ונטלי אהרון,https://feeds.transistor.fm/valuebell-dudu-aharon
7,32621,לנצח את הפסיכומטרי - תמיר עגיב,https://feeds.transistor.fm/valuebell-psychome...
8,31438,ליגלייז - פודקאסט קנאביס,https://feeds.transistor.fm/valuebell-legalize
9,43328,לא בטעות ולא במקרה || אלאיה הוף,https://feeds.transistor.fm/valuebell-alayah-hoff


In [ ]:
episodes_df.columns

Index(['Show ID', 'Show Title', 'Episode ID', 'Title', 'Number', 'Season',
       'Status', 'Published At', 'Created At', 'Duration (seconds)', 'Type'],
      dtype='object')

In [ ]:
import pandas as pd

# Rename the DataFrame columns to match the SQL table column names
shows.columns = ['show_id', 'title', 'rss_feed']

# Step 1: Create a unique combination of 'Show ID' and 'Show Title' from episodes_df
episodes_unique = episodes_df[['Show ID', 'Show Title']].drop_duplicates()

# Step 2: Check which rows in episodes_unique don't exist in shows
# Merge 'episodes_unique' with 'shows' based on 'Show ID' and 'Show Title'
merged_episodes_to_shows = pd.merge(episodes_unique, shows[['show_id', 'title']],
                                    left_on=['Show ID', 'Show Title'],
                                    right_on=['show_id', 'title'],
                                    how='left', indicator=True)

# Rows that exist in episodes_unique but not in shows
episodes_not_in_shows = merged_episodes_to_shows[merged_episodes_to_shows['_merge'] == 'left_only']
print("Rows in episodes_unique but not in shows:")
print(episodes_not_in_shows[['Show ID', 'Show Title']])

# Step 3: Check which rows in shows don't exist in episodes_unique
# Merge 'shows' with 'episodes_unique' based on 'Show ID' and 'Show Title'
merged_shows_to_episodes = pd.merge(shows[['show_id', 'title']], episodes_unique,
                                    left_on=['show_id', 'title'],
                                    right_on=['Show ID', 'Show Title'],
                                    how='left', indicator=True)

# Rows that exist in shows but not in episodes_unique
shows_not_in_episodes = merged_shows_to_episodes[merged_shows_to_episodes['_merge'] == 'left_only']
print("Rows in shows but not in episodes_unique:")
print(shows_not_in_episodes[['show_id', 'title']])

Rows in episodes_unique but not in shows:
Empty DataFrame
Columns: [Show ID, Show Title]
Index: []
Rows in shows but not in episodes_unique:
    show_id                                              title
24    53532  The Boring Sidee of Startups - החלק המשעמם של ...


In [ ]:
import pandas as pd

# Function to compute max and average length for each string column
def compute_max_avg_length(df, df_name):
    print(f"Results for DataFrame: {df_name}")

    # Iterate through each column in the DataFrame
    for column in df.columns:
        if df[column].dtype == 'object':  # Only work with object (string) columns
            # Remove null values from the column
            non_null_values = df[column].dropna()

            # Calculate the length of each string
            lengths = non_null_values.apply(len)

            # Calculate max and average length
            if not lengths.empty:
                max_length = lengths.max()
                avg_length = lengths.mean()
                print(f"Column: {column} | Max length: {max_length} | Average length: {avg_length:.2f}")
            else:
                print(f"Column: {column} has no data to calculate.")
        else:
            print(f"Column: {column} is not a string column.")

    print("\n")

# Example DataFrames: shows, episodes_df, downloads_df
# Assuming you already have the shows, episodes_df, and downloads_df created

# Call the function for each DataFrame
compute_max_avg_length(shows, "shows")
compute_max_avg_length(episodes_df, "episodes_df")
compute_max_avg_length(downloads_df, "downloads_df")


Results for DataFrame: shows
Column: Show ID is not a string column.
Column: Title | Max length: 52 | Average length: 24.38
Column: Feed URL | Max length: 58 | Average length: 47.76


Results for DataFrame: episodes_df
Column: Show ID is not a string column.
Column: Show Title | Max length: 41 | Average length: 22.73
Column: Episode ID is not a string column.
Column: Title | Max length: 133 | Average length: 37.56
Column: Number is not a string column.
Column: Season is not a string column.
Column: Status | Max length: 9 | Average length: 8.36
Column: Published At is not a string column.
Column: Created At is not a string column.
Column: Duration (seconds) is not a string column.
Column: Type | Max length: 7 | Average length: 4.09


Results for DataFrame: downloads_df
Column: Episode ID is not a string column.
Column: Episode Title | Max length: 133 | Average length: 43.82
Column: Show ID is not a string column.
Column: Download Date is not a string column.
Column: Downloads is not a s

# Insertion to Cloud SQL

In [ ]:
!pip install pymysql sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
!curl ifconfig.me

35.229.67.66

In [ ]:
from sqlalchemy import create_engine

# Replace these variables with your actual credentials
user = 'Tobaly'
password = 'Valueinsight2024'
database = 'transistor'
instance_connection_name = 'deep-voyage-437218-k1:me-west1:instance1'

# For connecting via public IP
host = '34.165.234.180'  # If using a public IP

# Creating connection string
connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)


In [ ]:
# Inserting to shows
shows.to_sql('shows', con=engine, if_exists='append', index=False)


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '37413' for key 'shows.PRIMARY'")
[SQL: INSERT INTO shows (show_id, title, rss_feed) VALUES (%(show_id)s, %(title)s, %(rss_feed)s)]
[parameters: [{'show_id': 37413, 'title': ' רונית רמר מספרת ׳סיפורי סיפורים׳', 'rss_feed': 'https://feeds.transistor.fm/valuebell-ronit-remer'}, {'show_id': 44556, 'title': 'The Business Mindset', 'rss_feed': 'https://feeds.transistor.fm/the-business-mindset'}, {'show_id': 32847, 'title': 'מנהלים ועובדים על זה || עו״ד יניב אופק', 'rss_feed': 'https://feeds.transistor.fm/yaniv-ofek'}, {'show_id': 36996, 'title': 'סוד המצוינות הישראלי - גיא קצוביץ', 'rss_feed': 'https://feeds.transistor.fm/valuebell-guy-book'}, {'show_id': 50986, 'title': 'דואט | דודו ונטלי אהרון', 'rss_feed': 'https://feeds.transistor.fm/valuebell-dudu-aharon'}, {'show_id': 32621, 'title': 'לנצח את הפסיכומטרי - תמיר עגיב', 'rss_feed': 'https://feeds.transistor.fm/valuebell-psychometric'}, {'show_id': 31438, 'title': 'ליגלי

In [ ]:
# Inserting to episodes
episodes_df.to_sql('episodes', con=engine, if_exists='append', index=False)


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '34.165.234.180' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Inserting to downloads
downloads_df.to_sql('downloads', con=engine, if_exists='append', index=False)

# Archive

In [ ]:
try:
    import feedparser
except ImportError:
    import os
    os.system('pip install feedparser')
    import feedparser

import pandas as pd

# Assume 'shows' is your DataFrame containing Feed URLs and Show Titles
episodes_list = []

# Iterate through each show and parse its RSS feed
for index, row in shows.iterrows():
    feed_url = row['Feed URL']
    show_title = row['Title']  # Show Title from the shows DataFrame
    feed = feedparser.parse(feed_url)

    for entry in feed.entries:
        episode_id = entry.id  # 'id' maps to the <guid> tag
        title = entry.title
        published_at = entry.published
        episode_type = entry.get('itunes_episodetype', 'unknown')  # Extract episode type
        duration = entry.get('itunes_duration', 'unknown')  # Extract episode duration
        episode_number = entry.get('podcast_episode', 'unknown')  # Extract episode number

        episodes_list.append({
            "Show ID": row['Show ID'],
            "Show Title": show_title,  # Add Show Title here
            "Episode ID": episode_id,
            "Title": title,
            "Published At": published_at,
            "Episode Type": episode_type,  # Add episode type
            "Duration": duration,  # Add episode duration
            "Episode Number": episode_number  # Add episode number
        })

# Create a DataFrame for the episodes
episodes = pd.DataFrame(episodes_list)

# Display the DataFrame
print(episodes)


    Show ID                      Show Title  \
0     32621  לנצח את הפסיכומטרי - תמיר עגיב   
1     32621  לנצח את הפסיכומטרי - תמיר עגיב   
2     32621  לנצח את הפסיכומטרי - תמיר עגיב   
3     32621  לנצח את הפסיכומטרי - תמיר עגיב   
4     32621  לנצח את הפסיכומטרי - תמיר עגיב   
..      ...                             ...   
271   31732         Digital Health Business   
272   31732         Digital Health Business   
273   31732         Digital Health Business   
274   31732         Digital Health Business   
275   31732         Digital Health Business   

                               Episode ID  \
0    c42a54de-463a-43d9-9cfd-6b7f34b1accc   
1    bb96e5c0-9b3c-4b7c-a737-656892aedd25   
2    96c8b87d-5b15-4ff1-9a5d-82a01470b4a0   
3    af364843-919f-4d70-b068-65f4178b5447   
4    891da0d8-fc05-47ed-b515-4f83f8f98d3f   
..                                    ...   
271  0ba1e802-45b7-493a-8eb2-42fcdd209e7a   
272  ec8d5df9-299d-4653-8832-ec5cb1b0d10d   
273  b8dffd2f-98ef-4e27-9512-3

In [ ]:
import feedparser

# Example RSS feed URL (replace with a real feed URL)
feed_url = "https://feeds.transistor.fm/f7a3f0b3-ed39-4312-98e7-b2c20d0b1d18"

# Parse the feed
feed = feedparser.parse(feed_url)

# Get the first entry (episode)
first_entry = feed.entries[0]

# Print all available fields for this episode
for key in first_entry.keys():
    print(f"{key}: {first_entry[key]}")


title: פרק 10: תקועים בחלל, הולכים בחלל, ומפנים את הזבל
title_detail: {'type': 'text/plain', 'language': None, 'base': 'https://feeds.transistor.fm/f7a3f0b3-ed39-4312-98e7-b2c20d0b1d18', 'value': 'פרק 10: תקועים בחלל, הולכים בחלל, ומפנים את הזבל'}
itunes_episode: 10
podcast_episode: 10
itunes_title: פרק 10: תקועים בחלל, הולכים בחלל, ומפנים את הזבל
itunes_episodetype: full
id: 9506263e-c439-45c3-8735-406ce116d5b6
guidislink: False
links: [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://share.transistor.fm/s/7a740491'}, {'length': '93632244', 'type': 'audio/mpeg', 'href': 'https://media.transistor.fm/7a740491/25ea0829.mp3', 'rel': 'enclosure'}]
link: https://share.transistor.fm/s/7a740491
summary: החללית סטארליינר חוזרת לכדור הארץ בידיים ריקות, ללא האסטרונאוטים שאיתם היא הגיעה לתחנת החלל – מה זה אומר לגבי עתיד הטיסות המסחריות המאוישות? ספייס אקס משגרת אנשים שאינם אסטרונאוטים במקצועם לאחת המשימות הנועזות בהיסטוריה – למה נועדה משימת פולאריס דון? איך נאס"א ממשיכה להכניס חברות מסח